In [1]:
exec(open('file_reader.py').read())
#df.head()

generating features..

 498293668655423488

 498486826269548545

 500280249629036544

 500298588992593920

 524923293711998976

 524936793633083394

 524941720249978880

 524948206023880704

 524961721744900097

 544274544174071809

 544294893146091520

 544315472075042818

 552788945017516032

 553480082996879360

 553553288625672192

 553561170637238272

 580352273001410560

 581153923987206146

 581290271997968384

 581359544682614784

 758159624122097664

 763098277986209792

 764927075522260992

 767725956706414592

 768859780240773121

 769988636754505729

 774991078265094144

 775057555865206784

 498293668655423488

 498486826269548545

 500280249629036544

 500298588992593920

 524923293711998976

 524936793633083394

 524941720249978880

 524948206023880704

 524961721744900097

 544274544174071809

 544294893146091520

 544315472075042818

 552788945017516032

 553480082996879360

 553553288625672192

 553561170637238272

 580352273001410560

 581153923987206146

 5812902719

In [ ]:
import sys
sys.path.insert(1, "./feature-extraction/twitter-features")
from EmbedExtractor import EmbedExtractor
from VulgarExtractor import VulgarExtractor

ee = EmbedExtractor()

In [ ]:
wordlist = VulgarExtractor.vulgarWords("badwords.txt") 
dftext = df[['text']]
result = dftext.applymap(lambda x: VulgarExtractor.containsVulgar(x,wordlist))
df['isVulgar'] = result


In [151]:
#word embeddings must be generated before POS
word_embeddings =

from TwitterParser import TwitterParser
textlist = [txt.replace('\n','') for txt in df['text'].tolist()]
tagged_sents = TwitterParser.tag(textlist)
df['POS'] = tagged_sents
#df.head()



,classification,context_path,has_context,text,topic,opinion,isVulgar,POS
580319078155468800,true,NaN,0,Germanwings Airbus A320 crashes in French Alps...,true,0,False,"[(Germanwings, ^), (Airbus, ^), (A320, ^), (cr..."
580319184652890113,false,NaN,0,BREAKING: 148 passengers were on board #German...,false,0,False,"[(BREAKING, N), (:, ,), (148, $), (passengers,..."
580320684305416192,unverified,NaN,0,Accident aircraft looks to be Germanwings (Air...,unverified,0,False,"[(Accident, N), (aircraft, N), (looks, V), (to..."
580321156508577792,false,NaN,0,Now hearing 148 passengers + crew on board the...,false,0,False,"[(Now, R), (hearing, V), (148, $), (passengers..."
580322453928431617,false,NaN,0,German Wings airline tweeting now about report...,false,0,False,"[(German, A), (Wings, N), (airline, N), (tweet..."


In [152]:
processed_sents = []
for tagged_sent in df['POS']:
    processed_words = []
    for word, tag in tagged_sent:
        if tag == 'U':
            processed_words.append('someurl')
        elif tag == '@':
            processed_words.append('@someuser')
        else:
            processed_words.append(word)
    sent = ' '.join(processed_words)
    processed_sents.append(sent)
df['text'] = processed_sents

In [171]:
df.head()

580319078155468800    False
580319184652890113    False
580320684305416192    False
580321156508577792    False
580322453928431617    False
580323060533764097    False
580324027715063808    False
580325090367315968    False
580326222107951104    False
580331561398108160    False
580332109782466561    False
580333763512705025    False
580333909008871424    False
580339547269144576    False
580339825649291264    False
580340476949086208    False
580348081100734464    False
580360165540642816    False
580371845997682688    False
580882341880446977    False
581047170637381632    False
581063377226637312    False
581293286268129280    False
581386094337474560    False
581473088249958400    False
Name: containsEmoji, dtype: bool

In [154]:
def normalize(column_name):
    std = df[column_name].std()
    norm_col = df[column_name].apply(lambda x: x - std)
    df[column_name] = norm_col

In [155]:
# import TwitterParser features
word_counts = [TwitterParser.word_count(tagged_line) for tagged_line in df['POS']]
pos_count_list = [TwitterParser.pos_counts(tagged_line) for tagged_line in df['POS']]
contains_adjs = [TwitterParser.contains_adjectives(tagged_line) for tagged_line in df['POS']]
contains_urls = [TwitterParser.contains_url(tagged_line) for tagged_line in df['POS']]
contains_emojis = [TwitterParser.contains_emoji(tagged_line) for tagged_line in df['POS']]
contains_abbrevs = [TwitterParser.contains_abbreviation(tagged_line) for tagged_line in df['POS']]

# get word count and normalize
df['wordCount'] = word_counts
normalize('wordCount')

# get an indexed list of pos tag counts
df['posCounts'] = pos_count_list

# get binary features
df['containsAdjective'] = contains_adjs
df['containsURL'] = contains_urls
df['containsEmoji'] = contains_emojis
df['containsAbbreviation'] = contains_abbrevs

df.head()

,classification,context_path,has_context,text,topic,opinion,isVulgar,POS,wordCount,posCounts,containsAdjective,containsURL,containsEmoji,containsAbbreviation
580319078155468800,true,NaN,0,Germanwings Airbus A320 crashes in French Alps...,true,0,False,"[(Germanwings, ^), (Airbus, ^), (A320, ^), (cr...",4.555903,"[0, 0, 0, 6, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 0, ...",False,True,False,False
580319184652890113,false,NaN,0,BREAKING : 148 passengers were on board #Germa...,false,0,False,"[(BREAKING, N), (:, ,), (148, $), (passengers,...",11.555903,"[3, 0, 0, 4, 0, 0, 0, 3, 1, 0, 0, 2, 2, 0, 0, ...",True,True,False,False
580320684305416192,unverified,NaN,0,Accident aircraft looks to be Germanwings ( Ai...,unverified,0,False,"[(Accident, N), (aircraft, N), (looks, V), (to...",11.555903,"[5, 0, 0, 4, 0, 0, 0, 2, 0, 0, 0, 0, 2, 1, 0, ...",False,True,False,False
580321156508577792,false,NaN,0,Now hearing 148 passengers + crew on board the...,false,0,False,"[(Now, R), (hearing, V), (148, $), (passengers...",13.555903,"[4, 0, 0, 2, 0, 0, 0, 3, 1, 1, 0, 1, 3, 1, 0, ...",True,False,False,False
580322453928431617,false,NaN,0,German Wings airline tweeting now about report...,false,0,False,"[(German, A), (Wings, N), (airline, N), (tweet...",12.555903,"[5, 0, 0, 2, 0, 0, 0, 2, 1, 1, 0, 0, 5, 0, 0, ...",True,True,False,False


In [156]:
for i, tag in enumerate(TwitterParser.tagset):
    tag_counts = []
    for pos_counts in df['posCounts']:
        tag_counts.append(pos_counts[i])
    column_name = 'num_' + tag
    df[column_name] = tag_counts
    normalize(column_name)


In [157]:
df.loc[df.classification == 'true', 'classification'] = 1
df.loc[df.classification == 'false', 'classification'] = 0
df.loc[df.classification == 'unverified', 'classification'] = 2


attributes = []
# getting the labels
# You have to comment this out if you want only tweet ID to be in the features. 
# Note that by doing this, you will screw up the simple/tr,simple/dev test located after this


attributes = ['isVulgar', 'containsAdjective', 'containsURL', 'containsEmoji', 'containsAbbreviation', 'wordCount']
for tag in TwitterParser.tagset:
    attributes.append('num_' + tag)
    
dev_labels = df['classification']
dev_labels = [l for l in dev_labels]
dev_labels = np.array(dev_labels)

# getting the values as a list of lists
dev_values = df[attributes].values.tolist()


#Below puts the tweet ID as a feature. Comment this out if you aren't using tweetID
###for i,index in enumerate(df.index):
###    dev_values[i].append(int(index))


dev_values = np.array(dev_values)
dev_values

array([[  0.00000000e+00,   0.00000000e+00,   1.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   4.55590279e+00,
         -2.03469899e+00,  -4.08248290e-01,   0.00000000e+00,
          4.09649446e+00,   0.00000000e+00,  -2.00000000e-01,
          0.00000000e+00,  -1.15993937e-02,  -7.78888096e-01,
         -5.00000000e-01,   0.00000000e+00,  -9.00000000e-01,
          4.75751551e-01,  -4.58257569e-01,  -3.31662479e-01,
          0.00000000e+00,   0.00000000e+00,  -5.68624070e-01,
         -4.08248290e-01,  -4.00000000e-01,   4.02784238e-01,
          0.00000000e+00,  -8.40634681e-01,  -2.13541565e+00,
         -2.00000000e-01],
       [  0.00000000e+00,   1.00000000e+00,   1.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   1.15559028e+01,
          9.65301005e-01,  -4.08248290e-01,   0.00000000e+00,
          2.09649446e+00,   0.00000000e+00,  -2.00000000e-01,
          0.00000000e+00,   1.98840061e+00,   2.21111904e-01,
         -5.00000000e-01,   0.00000000e+00,

In [158]:
import classifiers

# note predict_proba() gets probabilities for all 3 labels
#... and decision_tree_classifier uses decision_function() instead of predict_proba()... weird sklearn quirk
# NOTE: This is where you change the classifier type. Can pick from [naive_bayes, svm_classifier, decision_tree_classifier]
predictions, probabilities = classifiers.svm_classifier(dev_values, dev_labels, dev_values)

ps = []
for i, p in enumerate(predictions):
    if p == 0:
        ps.append('false')
    if p == 1:
        ps.append('true')
    if p == 2:
        ps.append('unverified')
    

# creates pairings of the prediction and the probability of the prediction
pred_probs_pairs = [[ps[i], probabilities[i][predictions[i]]] for i in range(len(predictions))]  

#now we make a dictionary of tweetID to the pred_probs_pairs
pred_dict = {index:pred_probs_pairs[i] for i,index in enumerate(df.index)}
pred_dict

{'580319078155468800': ['true', 0.95746996011498731],
 '580319184652890113': ['false', 0.91892254283776831],
 '580320684305416192': ['unverified', 0.32277503893861303],
 '580321156508577792': ['false', 0.94151205147246642],
 '580322453928431617': ['false', 0.7848577090372737],
 '580323060533764097': ['true', 0.86898159178682643],
 '580324027715063808': ['true', 0.48270640684194371],
 '580325090367315968': ['unverified', 0.48202942159192241],
 '580326222107951104': ['false', 0.49484034035034097],
 '580331561398108160': ['false', 0.83898892614722409],
 '580332109782466561': ['true', 1.485489419334052],
 '580333763512705025': ['false', 0.84365031964414494],
 '580333909008871424': ['true', 0.53162708465152719],
 '580339547269144576': ['false', 0.74389842527095906],
 '580339825649291264': ['false', 0.066571660462573179],
 '580340476949086208': ['false', 0.58048486709082114],
 '580348081100734464': ['true', 0.29082590644303236],
 '580360165540642816': ['false', 0.54334329295454309],
 '580371

In [159]:
with open('output/classifier_output/test.json', 'w') as outfile:
    json.dump(pred_dict, outfile)

In [160]:
## python3 scorer/score.py semeval2017-task8-dataset/traindev/rumoureval-subtaskB-dev.json output/classifier_output/test.json


## Training on simple/tr, testing on simple/dev


In [174]:
# Simply doing all of the transformations we did in the first few cells

tr_df = pd.read_pickle('output/simple/train_data_simple.pickle')

sys.path.insert(1, "./feature-extraction/twitter-features")

wordlist = VulgarExtractor.vulgarWords("badwords.txt") 
dftext = tr_df[['text']]
result = dftext.applymap(lambda x: VulgarExtractor.containsVulgar(x,wordlist))
tr_df['isVulgar'] = result

textlist = [txt.replace('\n','') for txt in tr_df['text'].tolist()]
tagged_sents = TwitterParser.tag(textlist)
tr_df['POS'] = tagged_sents

processed_sents = []
for tagged_sent in tr_df['POS']:
    processed_words = []
    for word, tag in tagged_sent:
        if tag == 'U':
            processed_words.append('someurl')
        elif tag == '@':
            processed_words.append('@someuser')
        else:
            processed_words.append(word)
    sent = ' '.join(processed_words)
    processed_sents.append(sent)
tr_df['text'] = processed_sents

word_counts = [TwitterParser.word_count(tagged_line) for tagged_line in tr_df['POS']]
pos_count_list = [TwitterParser.pos_counts(tagged_line) for tagged_line in tr_df['POS']]
contains_adjs = [TwitterParser.contains_adjectives(tagged_line) for tagged_line in tr_df['POS']]
contains_urls = [TwitterParser.contains_url(tagged_line) for tagged_line in tr_df['POS']]
contains_emojis = [TwitterParser.contains_emoji(tagged_line) for tagged_line in tr_df['POS']]
contains_abbrevs = [TwitterParser.contains_abbreviation(tagged_line) for tagged_line in tr_df['POS']]

tr_df['wordCount'] = word_counts
tr_df['posCounts'] = pos_count_list
tr_df['containsAdjective'] = contains_adjs
tr_df['containsURL'] = contains_urls
tr_df['containsEmoji'] = contains_emojis
tr_df['containsAbbreviation'] = contains_abbrevs

for i, tag in enumerate(TwitterParser.tagset):
    tag_counts = []
    for pos_counts in tr_df['posCounts']:
        tag_counts.append(pos_counts[i])
    column_name = 'num_' + tag
    tr_df[column_name] = tag_counts

tr_df.head()

,classification,context_path,has_context,text,topic,isVulgar,POS,wordCount,posCounts,containsAdjective,...,num_X,num_Y,num_#,num_@,num_~,num_U,num_E,num_$,"num_,",num_G
498280126254428160,unverified,NaN,0,Mike Brown was staying with his grandmother fo...,ferguson,False,"[(Mike, ^), (Brown, ^), (was, V), (staying, V)...",15,"[3, 1, 0, 2, 0, 0, 0, 3, 0, 0, 0, 3, 3, 0, 0, ...",False,...,0,0,1,0,0,0,0,0,2,0
498430783699554305,true,NaN,0,Witness : Police allegedly stopped Mike Brown ...,ferguson,False,"[(Witness, N), (:, ,), (Police, N), (allegedly...",14,"[3, 1, 0, 2, 0, 0, 0, 3, 0, 1, 0, 0, 4, 0, 0, ...",False,...,0,0,1,0,0,1,0,0,2,0
499366666300846081,unverified,NaN,0,Line of police cars with high beams on greets ...,ferguson,False,"[(Line, N), (of, P), (police, N), (cars, N), (...",20,"[6, 0, 0, 1, 0, 1, 0, 5, 1, 0, 0, 1, 4, 0, 1, ...",True,...,0,0,0,0,0,1,0,0,3,0
499368931367608320,unverified,NaN,0,"Currently the #FoxNews website has zero , repe...",ferguson,False,"[(Currently, R), (the, D), (#FoxNews, ^), (web...",20,"[7, 0, 0, 3, 0, 0, 0, 1, 1, 2, 0, 2, 3, 0, 0, ...",True,...,0,0,0,0,0,0,0,1,4,0
499456140044824576,unverified,./semeval2017-task8-dataset/rumoureval-data/fe...,1,St. Louis Co Police tell me ofcr shot a man wh...,ferguson,False,"[(St., ^), (Louis, ^), (Co, ^), (Police, ^), (...",26,"[5, 3, 0, 6, 0, 0, 0, 2, 2, 0, 0, 1, 5, 1, 0, ...",True,...,0,0,1,0,0,0,0,1,1,0


In [162]:
# Changes "true"/"false"/"unverified" to numeric values, just like the in the early cells

tr_df.loc[tr_df.classification == 'true', 'classification'] = 1
tr_df.loc[tr_df.classification == 'false', 'classification'] = 0
tr_df.loc[tr_df.classification == 'unverified', 'classification'] = 2
# getting the labels

attributes = ['isVulgar', 'containsAdjective', 'containsURL', 'containsEmoji', 'containsAbbreviation', 'wordCount']
for tag in TwitterParser.tagset:
    attributes.append('num_' + tag)

tr_labels = tr_df['classification']
tr_labels = [l for l in tr_labels]
tr_labels = np.array(tr_labels)


# getting the values as a list of lists
tr_values = tr_df[attributes].values.tolist()
tr_values = np.array(tr_values)

In [167]:
predictions, probabilities = classifiers.decision_tree_classifier(tr_values, tr_labels, dev_values)
print(probabilities)
ps = []
for i, p in enumerate(predictions):
    if p == 0:
        ps.append('false')
    if p == 1:
        ps.append('true')
    if p == 2:
        ps.append('unverified')
    

# creates pairings of the prediction and the probability of the prediction
pred_probs_pairs = [[ps[i], probabilities[i][predictions[i]]] for i in range(len(predictions))]  


pred_dict = {index:pred_probs_pairs[i] for i,index in enumerate(df.index)}


pred_dict

[[ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]]


{'580319078155468800': ['false', 1.0],
 '580319184652890113': ['false', 1.0],
 '580320684305416192': ['true', 1.0],
 '580321156508577792': ['true', 1.0],
 '580322453928431617': ['unverified', 1.0],
 '580323060533764097': ['false', 1.0],
 '580324027715063808': ['false', 1.0],
 '580325090367315968': ['true', 1.0],
 '580326222107951104': ['true', 1.0],
 '580331561398108160': ['unverified', 1.0],
 '580332109782466561': ['unverified', 1.0],
 '580333763512705025': ['true', 1.0],
 '580333909008871424': ['true', 1.0],
 '580339547269144576': ['unverified', 1.0],
 '580339825649291264': ['false', 1.0],
 '580340476949086208': ['false', 1.0],
 '580348081100734464': ['unverified', 1.0],
 '580360165540642816': ['unverified', 1.0],
 '580371845997682688': ['false', 1.0],
 '580882341880446977': ['true', 1.0],
 '581047170637381632': ['false', 1.0],
 '581063377226637312': ['unverified', 1.0],
 '581293286268129280': ['false', 1.0],
 '581386094337474560': ['unverified', 1.0],
 '581473088249958400': ['false'

In [168]:
with open('output/classifier_output/tr_test.json', 'w') as outfile:
    json.dump(pred_dict, outfile)
    
#test with:
### python3 scorer/score.py semeval2017-task8-dataset/traindev/rumoureval-subtaskB-dev.json output/classifier_output/tr_test.json